<a href="https://colab.research.google.com/github/fduartej/forecasting-food/blob/main/Forecasting_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Forecasting Food**

Resumen

El presente trabajo tiene como objetivo la creacion de un modelo de regresion de ML para el prediccion de la comida (forecasting food)

In [1]:
# imports
from nltk.corpus import stopwords 
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

ModuleNotFoundError: No module named 'nltk'